In [166]:
%reload_ext autoreload
%autoreload 2
import csv
from CIOGame import *
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, ensemble
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from gurobipy import *

# Reports in previous semesters

In [ ]:
file = 'CMO_Controlling-Tool.xlsx'
# Load spreadsheet
xl = pd.ExcelFile(file)
df=xl.parse('Marketing KPIs').drop(list(range(10,18)))
df=df.set_index([list(df.iloc[:,0])]).iloc[:,1:].transpose().drop(['Prozessqualität'],axis=1)
df

In [ ]:
cio_game.correlation_matrix(df)

In [ ]:
df_corr=df.corr()
df_corr

In [ ]:
df_corr.abs().sort_values(by=['     Global'],ascending=False)#.drop(["Marktanteil (New Savings)","Kundenzufriedenheit"])

# blablathings

In [173]:
cio_game=CIOGame(5)
df_cmo=cio_game.read_from_exel().T

Created CIOGame Object for period 5


In [174]:
df_cmo

,Car Financing Loans,Customer Savings,Requests to be processed LowPriceCars,Requests to be processed MidPriceCars,Requests to be processed HighPriceCars,Contracts succeeded LowPriceCars,Contracts succeeded MidPriceCars,Contracts succeeded HighPriceCars,Service Transactions to be processed loans,Service Transactions succeeded loans,...,Contracts succeeded savings,Service Transactions succeeded savings,Customer Satisfaction,Marketing Efficiency for product Car Financing Loans,Marketing Efficiency for product Savings Account,Marketing Expenditures Global,Marketing Expenditures Product Loans,Marketing Expenditures Product Savings,Interest Rate Car Financing Loans,Interest Rate Customer Savings
0,1.80620e+09,1.08427e+09,59391.8,74239.8,14848,18460.7,23075.9,4615.2,216749,173290,...,136564,156518,0.8332,0.7295,0.6915,4500000,2220000,1200000,0.12,0.03
1,2.59591e+09,9.6252e+08,56685.5,70856.9,14171.4,39239.7,49049.6,9809.9,311510,298466,...,172480,135516,0.8492,0.6938,0.6576,4500000,2020000,1400000,0.12,0.035
2,3.38719e+09,1.13823e+09,62877.9,78597.4,15719.5,41754.9,52193.6,10438.7,406463,327376,...,308383,162807,0.8681,0.7884,0.768,4500000,2220000,1200000,0.123,0.035
3,3.73218e+09,2.02188e+09,46496.4,58120.5,11624.1,35495.5,44369.4,8873.9,447862,425422,...,772732,285108,0.8929,0.9085,0.9026,3300000,1100000,900000,0.13,0.032
4,4.4754e+09,3.03828e+09,54842.7,68553.3,13710.7,50091.1,62613.9,12522.8,537047,524853,...,1.04885e+06,428043,0.9269,0.9689,0.9684,3600000,1200000,1440000,0.137,0.03
5,5.574e+09,5.43358e+09,63389,79236.3,15847.3,60304.8,75381,15076.2,668880,651918,...,2.07151e+06,668233,0.9663,0.9854,0.9854,4500000,1500000,5000000,0.147,0.023


# test

In [ ]:
global_expenditure=np.array([4500000,4500000,4500000,3300000,3600000,4500000])

savings_intrests_rate=np.array([0.0300,0.0350,0.0350,0.0320,0.03,0.023])
savings_expenditure=np.array([1200000,1400000,1200000,900000,1440000,5000000])
savings_marketing_efficiency=np.array([0.6915,0.6576,0.7680,0.9026,0.9684,0.9864])

loans_intrests_rate=np.array([0.12,0.12,0.123,0.13,0.137,0.147])
loans_expenditure=np.array([2220000,2020000,2220000,1100000,1200000,1500000])
loans_marketing_efficiency=np.array([0.7295,0.6938,0.7884,0.9085,0.9689,0.9854])

savings_requests=np.array([902795.10,951090.40,993601.10,837795.40,1114117.8,2376077.1 ])
loans_requests=np.array([148479.60,141713.80 ,157194.80 ,116241.00,137106.70,158472.6])
#loans_requests=np.array([74239.80 ,70856.90 ,78597.40 ,58120.50 ,68553.30 ,79236.3])

df_savings=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "savings_expenditure":savings_expenditure,
    "savings_intrests_rate":savings_intrests_rate,
    "savings_marketing_efficiency":savings_marketing_efficiency,
    "savings_requests":savings_requests})[:-1]
df_loans=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "loans_expenditure":loans_expenditure,
    "loans_intrests_rate":loans_intrests_rate,
    "loans_marketing_efficiency":loans_marketing_efficiency,
    "loans_requests":loans_requests})[:-1]

In [ ]:
df_savings

In [ ]:
df_loans

In [ ]:
def find_best_regr(X,y,df_pre,y_pre):
    diff=[]
    regr_list=[]
    data_size=len(y)
    for i in range(data_size):
        a=list(range(data_size))
        del a[i]
        X_train=X.iloc[a]
        X_test=X.iloc[i].values.reshape(1,-1)
        y_train=y[a]
        y_test=y[i]

        regr = linear_model.LinearRegression()
        regr.fit(X_train,y_train)
        regr_list.append(regr)
        diff.append(regr.predict(X_test)[0]/y_test)

    list_diff=list(abs(np.array(diff)-1))
    best_regr_index=list_diff.index(min(list_diff))
    best_regr=regr_list[best_regr_index]
    print(list_diff[best_regr_index])
    print(y_pre )
    print(best_regr.predict(df_pre)[0])
    return best_regr

In [ ]:
y_savings=df_savings["savings_requests"]
X_savings=df_savings.drop(["savings_requests"],axis=1)

df_savings_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[5000000],
    "savings_intrests_rate":[0.02],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_savings=find_best_regr(X_savings,y_savings,df_savings_pre,2376077.1)

In [ ]:
y_loans=df_loans["loans_requests"]
X_loans=df_loans.drop(["loans_requests"],axis=1)

df_loans_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[1500000],
    "savings_intrests_rate":[0.147],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_loans=find_best_regr(X_loans,y_loans,df_loans_pre,158472.6)

In [ ]:
best_regr_savings.coef_

In [ ]:
best_regr_loans.coef_